In [12]:
import numpy as np
import re
import json
import pandas as pd
import warnings
import csv
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
%load_ext autoreload
%autoreload 2
import label_improve as li

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Loading the data 
dataset_name = "banking77"
idx_to_label = json.load(open(f"label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
test_df = li.data_to_df(json.load(open(f"test.json", "r")))

In [15]:
test_df = test_df.sample(250, random_state=205)
eval_df = test_df.drop(columns=[ 'label', 'weak_labels'])

In [16]:
eval_df.to_csv(f'./{dataset_name}_eval.csv')

In [17]:
from openai import OpenAI
import os
import sys

API_KEY = ''


model_id = 'gpt-4o-2024-08-06' # replace with your model ID

os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [18]:
import openai
import pandas as pd

# Load your CSV file
df = pd.read_csv(f'{dataset_name}_eval.csv')


# Define the categories
categories = {
    "0": "activate_my_card",
    "1": "age_limit",
    "2": "apple_pay_or_google_pay",
    "3": "atm_support",
    "4": "automatic_top_up",
    "5": "balance_not_updated_after_bank_transfer",
    "6": "balance_not_updated_after_cheque_or_cash_deposit",
    "7": "beneficiary_not_allowed",
    "8": "cancel_transfer",
    "9": "card_about_to_expire",
    "10": "card_acceptance",
    "11": "card_arrival",
    "12": "card_delivery_estimate",
    "13": "card_linking",
    "14": "card_not_working",
    "15": "card_payment_fee_charged",
    "16": "card_payment_not_recognised",
    "17": "card_payment_wrong_exchange_rate",
    "18": "card_swallowed",
    "19": "cash_withdrawal_charge",
    "20": "cash_withdrawal_not_recognised",
    "21": "change_pin",
    "22": "compromised_card",
    "23": "contactless_not_working",
    "24": "country_support",
    "25": "declined_card_payment",
    "26": "declined_cash_withdrawal",
    "27": "declined_transfer",
    "28": "direct_debit_payment_not_recognised",
    "29": "disposable_card_limits",
    "30": "edit_personal_details",
    "31": "exchange_charge",
    "32": "exchange_rate",
    "33": "exchange_via_app",
    "34": "extra_charge_on_statement",
    "35": "failed_transfer",
    "36": "fiat_currency_support",
    "37": "get_disposable_virtual_card",
    "38": "get_physical_card",
    "39": "getting_spare_card",
    "40": "getting_virtual_card",
    "41": "lost_or_stolen_card",
    "42": "lost_or_stolen_phone",
    "43": "order_physical_card",
    "44": "passcode_forgotten",
    "45": "pending_card_payment",
    "46": "pending_cash_withdrawal",
    "47": "pending_top_up",
    "48": "pending_transfer",
    "49": "pin_blocked",
    "50": "receiving_money",
    "51": "Refund_not_showing_up",
    "52": "request_refund",
    "53": "reverted_card_payment?",
    "54": "supported_cards_and_currencies",
    "55": "terminate_account",
    "56": "top_up_by_bank_transfer_charge",
    "57": "top_up_by_card_charge",
    "58": "top_up_by_cash_or_cheque",
    "59": "top_up_failed",
    "60": "top_up_limits",
    "61": "top_up_reverted",
    "62": "topping_up_by_card",
    "63": "transaction_charged_twice",
    "64": "transfer_fee_charged",
    "65": "transfer_into_account",
    "66": "transfer_not_received_by_recipient",
    "67": "transfer_timing",
    "68": "unable_to_verify_identity",
    "69": "verify_my_identity",
    "70": "verify_source_of_funds",
    "71": "verify_top_up",
    "72": "virtual_card_not_working",
    "73": "visa_or_mastercard",
    "74": "why_verify_identity",
    "75": "wrong_amount_of_cash_received",
    "76": "wrong_exchange_rate_for_cash_withdrawal"
}
# Function to classify text
def classify_text(text):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are an expert in banking financial service domain, and you are good at classification."},
            {"role": "user", "content": f"Classify the following text into one of the categories: {categories}. Text: {text}, also provide the label number at the end of the sentences, with the exact sentence 'The label is : $number'"}
        ]
    )
    classification = response.choices[0].message.content
    return classification.strip()



# Apply the function to each line of text
df['Classification'] = df['text'].apply(classify_text)

# Save the results to a new CSV file
df.to_csv('classified_file.csv', index=False)


In [20]:
df['Classification'] = df['Classification'].str.extract(r'The label is\s*:\s*(\d+)')
 
df.to_csv('classified_file2.csv', index=False)


In [21]:
predictions_df = pd.read_csv('classified_file2.csv')

true_labels_df =  test_df

merged_df = pd.merge(predictions_df, true_labels_df, on='text')

merged_df['correct'] = merged_df['Classification'] == merged_df['label']

accuracy = merged_df['correct'].mean()
print(f"Accuracy: {accuracy:.2%}")

print(merged_df.head())

merged_df.to_csv('comparison_with_true_labels.csv', index=False)


Accuracy: 73.20%
   Unnamed: 0                                               text  \
0        1061  My bank statement doesn't show the cash deposi...   
1        2386                   How is the top-up card verified?   
2          49  I have one of your cards already, how do I lin...   
3        3072                     Am I able to get a card in EU?   
4          13                       still waiting on my new card   

   Classification  label                                        weak_labels  \
0             6.0      6  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1            71.0     71  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
2            13.0     13  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
3            24.0     24  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
4            11.0     11  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

   correct  
0     True  
1     True  
2     True  
3     True  
4     True  
